<a href="https://colab.research.google.com/github/srkapoor/FPL_Optimization/blob/main/FPL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [71]:
!pip install pulp

In [117]:
import pandas as pd
import numpy as np
import csv 
import os
import requests
import json
from bs4 import BeautifulSoup
import re
import codecs
import pandas as pd
import os
import datetime
import pandas as pd
from pulp import *
import sasoptpy as so
import random
import string
from subprocess import Popen, DEVNULL


In [6]:
def get_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Response was code " + str(response.status_code))
    html = response.text
    parsed_html = BeautifulSoup(html, 'html.parser')
    scripts = parsed_html.findAll('script')
    filtered_scripts = []
    for script in scripts:
        if len(script.contents) > 0:
            filtered_scripts += [script]
    return scripts

def get_epl_data():
    scripts = get_data("https://understat.com/league/EPL/2021")
    teamData = {}
    playerData = {}
    for script in scripts:
        for c in script.contents:
            split_data = c.split('=')
            data = split_data[0].strip()
            if data == 'var teamsData':
                content = re.findall(r'JSON\.parse\(\'(.*)\'\)',split_data[1])
                decoded_content = codecs.escape_decode(content[0], "hex")[0].decode('utf-8')
                teamData = json.loads(decoded_content)
            elif data == 'var playersData':
                content = re.findall(r'JSON\.parse\(\'(.*)\'\)',split_data[1])
                decoded_content = codecs.escape_decode(content[0], "hex")[0].decode('utf-8')
                playerData = json.loads(decoded_content)
    return playerData

def get_player_data(id):
    scripts = get_data("https://understat.com/player/" + str(id))
    #groupsData = {}
    matchesData = {}
    #shotsData = {}
    for script in scripts:
        for c in script.contents:
            split_data = c.split('=')
            data = split_data[0].strip()
            if data == 'var matchesData':
                content = re.findall(r'JSON\.parse\(\'(.*)\'\)',split_data[1])
                decoded_content = codecs.escape_decode(content[0], "hex")[0].decode('utf-8')
                matchesData = json.loads(decoded_content)
    df_matchesData = pd.json_normalize(matchesData)
    df_matchesData['id'] = id
    return df_matchesData

# def parse_epl_data(outfile_base):
#     matchesData = get_epl_data()
#     new_matches_data = []
#     for t,v in matchesData.items():
#         new_matches_data += [v]
#     for data in new_team_data:
#         team_frame = pd.DataFrame.from_records(data["history"])
#         team = data["title"].replace(' ', '_')
#         team_frame.to_csv(os.path.join(outfile_base, 'understat_' + team + '.csv'), index=False)
#     player_frame = pd.DataFrame.from_records(playerData)
#     player_frame.to_csv(os.path.join(outfile_base, 'understat_player.csv'), index=False)

range = [datetime.datetime.today().strftime('%Y-%m-%d'), 
(datetime.datetime.today() - datetime.timedelta(weeks = 6)).strftime('%Y-%m-%d')]
epl_data = pd.json_normalize(get_epl_data())[['id', 'player_name']]
player_data = pd.DataFrame()
for id in epl_data.id.unique():
  data = get_player_data(id)
  data = data[(data.date <= range[0]) & (data.date >= range[1])]
  data = data.merge(epl_data, on = ['id'], how = 'inner')
  player_data = player_data.append(data)
player_data

KeyboardInterrupt: ignored

In [ ]:
player_data[player_data.player_name == 'Nélson Semedo']

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_name


# New Section

In [73]:
def get_data(team_id, gw):
    # get FPL api data
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    
    # get FPL player and team data
    element_data = pd.DataFrame(fpl_data['elements'])
    team_data = pd.DataFrame(fpl_data['teams'])
    type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
    elements_team = pd.merge(element_data, team_data, left_on='team', right_on='id')
                        
    # get fplreview data and create final merged table
    review_data =  pd.read_csv('gdrive/MyDrive/fplreview_gw12_17.csv')
    review_data['review_id'] = review_data.index+1
    merged_data = pd.merge(elements_team, review_data, left_on='id_x', right_on='review_id')
    merged_data.set_index(['id_x'], inplace=True)
    next_gw = int(review_data.keys()[5].split('_')[0])
    
    # get my team data
    r = requests.get(f'https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/')
    picks_data = r.json()
    initial_squad = [i['element'] for i in picks_data['picks']]
    r = requests.get(f'https://fantasy.premierleague.com/api/entry/{team_id}/')
    general_data = r.json()
    itb = general_data['last_deadline_bank'] / 10
    
    return {
        'merged_data': merged_data,
        'elements_team': elements_team,
        'team_data' : team_data,
        'type_data' : type_data,
        'next_gw' : next_gw,
        'initial_squad' : initial_squad,
        'review_data': review_data,
        'itb': itb
    }

In [74]:
def get_transfer_history(team_id, last_gw):
    transfers = []
    # Reversing GW history until a chip is played or 2+ transfers were made
    for gw in range(last_gw, 0, -1):
        res = requests.get(f'https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/').json()
        transfer = res['entry_history']['event_transfers']
        chip = res['active_chip']

        transfers.append(transfer)
        if transfer > 1 or (chip is not None and (chip != '3xc' or chip != 'bboost')):
            break

    return transfers

def get_rolling(team_id, last_gw):
    transfers = get_transfer_history(team_id, last_gw)

    # Start from gw where last chip used or when hits were taken
    # Reset FT count
    rolling = 0
    for transfer in reversed(transfers):
        # Transfer logic
        rolling = min(max(rolling + 1 - transfer, 0), 1)

    return rolling, transfers[0]

In [26]:
def get_random_id(n):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(n))

def solve_decision_support(team_id, gw, options):
    """
    Solves decision support problem for FPL
    Parameters
    ----------
    team_id: integer
        FPL ID of the team to be optimized
    gw: integer
        Upcoming (next) gameweek
    options: dict
        Options for the FPL problem
    """

    # Data
    
    data = get_data(team_id, gw-1)
    ft = get_rolling(team_id, gw-1)[0] + 1
    merged_data = data['merged_data']
    team_data = data['team_data']
    type_data = data['type_data']
    next_gw = data['next_gw']
    initial_squad = data['initial_squad']
    itb = data['itb']

    # Read options
    horizon = options.get('horizon', min(3, 38-next_gw+1))
    objective = options.get('objective', 'regular')
    decay_base = options.get('decay_base', 0.84)
    nsols = options.get('number_of_solutions', 1)
    alternative_solution = options.get('alternative_solution', '1gw-buy')
    strategy = None

    problem_name = f'ds_h{horizon}_{get_random_id(5)}'

    # Sets
    players = merged_data.index.to_list()
    element_types = type_data.index.to_list()
    teams = team_data['name'].to_list()
    gameweeks = list(range(next_gw, next_gw+horizon))
    all_gw = [next_gw-1] + gameweeks

    # Model
    model = so.Model(name=problem_name)

    # Variables
    squad = model.add_variables(players, all_gw, name='squad', vartype=so.binary)
    lineup = model.add_variables(players, gameweeks, name='lineup', vartype=so.binary)
    captain = model.add_variables(players, gameweeks, name='captain', vartype=so.binary)
    vicecap = model.add_variables(players, gameweeks, name='vicecap', vartype=so.binary)
    transfer_in = model.add_variables(players, gameweeks, name='transfer_in', vartype=so.binary)
    transfer_out = model.add_variables(players, gameweeks, name='transfer_out', vartype=so.binary)
    in_the_bank = model.add_variables(all_gw, name='itb', vartype=so.continuous, lb=0)
    free_transfers = model.add_variables(all_gw, name='ft', vartype=so.integer, lb=1, ub=2)
    penalized_transfers = model.add_variables(gameweeks, name='pt', vartype=so.integer, lb=0)
    aux = model.add_variables(gameweeks, name='aux', vartype=so.binary)
    
    # Dictionaries
    lineup_type_count = {(t,w): so.expr_sum(lineup[p,w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    squad_type_count = {(t,w): so.expr_sum(squad[p,w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    sell_price = (merged_data['SV']).to_dict()
    buy_price = (merged_data['BV']).to_dict()
    sold_amount = {w: so.expr_sum(sell_price[p] * transfer_out[p,w] for p in players) for w in gameweeks}
    bought_amount = {w: so.expr_sum(buy_price[p] * transfer_in[p,w] for p in players) for w in gameweeks}
    points_player_week = {(p,w): merged_data.loc[p, f'{w}_Pts']    for p in players for w in gameweeks}
    squad_count = {w: so.expr_sum(squad[p, w] for p in players) for w in gameweeks}
    number_of_transfers = {w: so.expr_sum(transfer_out[p,w] for p in players) for w in gameweeks}
    number_of_transfers[next_gw-1] = 1
    transfer_diff = {w: number_of_transfers[w] - free_transfers[w] for w in gameweeks}

    # Initial conditions
    model.add_constraints((squad[p, next_gw-1] == 1 for p in initial_squad), name='initial_squad_players')
    model.add_constraints((squad[p, next_gw-1] == 0 for p in players if p not in initial_squad), name='initial_squad_others')
    model.add_constraint(in_the_bank[next_gw-1] == itb, name='initial_itb')
    model.add_constraint(free_transfers[next_gw-1] == ft, name='initial_ft')

    # Constraints
    model.add_constraints((squad_count[w] == 15 for w in gameweeks), name='squad_count')
    model.add_constraints((so.expr_sum(lineup[p,w] for p in players) == 11 for w in gameweeks), name='lineup_count')
    model.add_constraints((so.expr_sum(captain[p,w] for p in players) == 1 for w in gameweeks), name='captain_count')
    model.add_constraints((so.expr_sum(vicecap[p,w] for p in players) == 1 for w in gameweeks), name='vicecap_count')
    model.add_constraints((lineup[p,w] <= squad[p,w] for p in players for w in gameweeks), name='lineup_squad_rel')
    model.add_constraints((captain[p,w] <= lineup[p,w] for p in players for w in gameweeks), name='captain_lineup_rel')
    model.add_constraints((vicecap[p,w] <= lineup[p,w] for p in players for w in gameweeks), name='vicecap_lineup_rel')
    model.add_constraints((captain[p,w] + vicecap[p,w] <= 1 for p in players for w in gameweeks), name='cap_vc_rel')
    model.add_constraints((lineup_type_count[t,w] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']] for t in element_types for w in gameweeks), name='valid_formation')
    model.add_constraints((squad_type_count[t,w] == type_data.loc[t, 'squad_select'] for t in element_types for w in gameweeks), name='valid_squad')
    model.add_constraints((so.expr_sum(squad[p,w] for p in players if merged_data.loc[p, 'name'] == t) <= 3 for t in teams for w in gameweeks), name='team_limit')
    ## Transfer constraints
    model.add_constraints((squad[p,w] == squad[p,w-1] + transfer_in[p,w] - transfer_out[p,w] for p in players for w in gameweeks), name='squad_transfer_rel')
    model.add_constraints((in_the_bank[w] == in_the_bank[w-1] + sold_amount[w] - bought_amount[w] for w in gameweeks), name='cont_budget')
    ## Free transfer constraints
    model.add_constraints((free_transfers[w] == aux[w] + 1 for w in gameweeks), name='aux_ft_rel')
    model.add_constraints((free_transfers[w-1] - number_of_transfers[w-1] <= 2 * aux[w] for w in gameweeks), name='force_aux_1')
    model.add_constraints((free_transfers[w-1] - number_of_transfers[w-1] >= aux[w] + (-14)*(1-aux[w]) for w in gameweeks), name='force_aux_2')
    model.add_constraints((penalized_transfers[w] >= transfer_diff[w] for w in gameweeks), name='pen_transfer_rel')

    # Objectives
    gw_xp = {w: so.expr_sum(points_player_week[p,w] * (lineup[p,w] + captain[p,w] + 0.1*vicecap[p,w]) for p in players) for w in gameweeks}
    gw_total = {w: gw_xp[w] - 4 * penalized_transfers[w] for w in gameweeks}

    obj_dict = {
        'regular': -so.expr_sum(gw_total[w] for w in gameweeks),
        'decay': -so.expr_sum(gw_total[w] * pow(decay_base, w-next_gw) for w in gameweeks),
        'next_gw_regular': -gw_total[next_gw]
    }

    if isinstance(objective, list):
        strategy = options.get('multi_obj_strategy', 'weight-sum')
        if strategy == 'weight-sum':
            objw = options.get('multi_obj_weights', [0.5, 0.5])
            model.set_objective(sum(objw[i] * obj_dict[o] for (i,o) in enumerate(objective)), sense='N', name='multiple_objectives_ws')
        elif strategy == '2-step':
            model.set_objective(obj_dict[objective[0]], sense='N', name='2step_obj1')
    else:
        model.set_objective(obj_dict[objective], sense='N', name='single_objective')

    results = []

    for it in range(nsols):

        # Solve
        model.export_mps(f'gdrive/MyDrive/tmp/{problem_name}.mps')
        command = f'cbc gdrive/MyDrive/tmp/{problem_name}.mps solve solu gdrive/MyDrive/tmp/{problem_name}_sol.txt'
        process = Popen(command, shell=False) # add 'stdout=DEVNULL' for disabling logs
        process.wait()
        # Parsing
        for v in model.get_variables():
            v.set_value(0)
        with open(f'gdrive/MyDrive/tmp/{problem_name}_sol.txt', 'r') as f:
            for line in f:
                if 'objective value' in line:
                    continue
                words = line.split()
                var = model.get_variable(words[1])
                var.set_value(float(words[2]))

        if strategy == '2-step':
            obj1_val = -1 * obj_dict[objective[0]].get_value()
            obj2_val = -1 * obj_dict[objective[1]].get_value()
            print(f'Obj 1 value: {obj1_val}, Obj 2 value: {obj2_val}')
            tradeoff = options['multi_obj_tradeoff']
            obj1_lower_bound = obj1_val - tradeoff
            tradeoff_con = model.add_constraint(-1 * obj_dict[objective[0]] >= obj1_lower_bound, name='tradeoff_2s')
            model.set_objective(obj_dict[objective[1]], sense='N', name='2step_obj2')
            model.export_mps(f'gdrive/MyDrive/tmp/{problem_name}.mps')
            command = f'cbc gdrive/MyDrive/tmp{problem_name}.mps mips gdrive/MyDrive/{problem_name}_sol.txt solve solu gdrive/MyDrive/tmp/{problem_name}_sol.txt'
            # process = Popen(command, shell=False) # add 'stdout=DEVNULL' for disabling logs
            # process.wait()
            !{command}

            for v in model.get_variables():
                v.set_value(0)
            with open(f'gdrive/MyDrive/tmp/{problem_name}_sol.txt', 'r') as f:
                for line in f:
                    if 'objective value' in line:
                        continue
                    words = line.split()
                    var = model.get_variable(words[1])
                    var.set_value(float(words[2]))

        # DataFrame generation
        picks = []
        for w in gameweeks:
            for p in players:
                if squad[p,w].get_value() + transfer_out[p,w].get_value() > 0.5:
                    lp = merged_data.loc[p]
                    is_captain = 1 if captain[p,w].get_value() > 0.5 else 0
                    is_lineup = 1 if lineup[p,w].get_value() > 0.5 else 0
                    is_vice = 1 if vicecap[p,w].get_value() > 0.5 else 0
                    is_transfer_in = 1 if transfer_in[p,w].get_value() > 0.5 else 0
                    is_transfer_out = 1 if transfer_out[p,w].get_value() > 0.5 else 0
                    position = type_data.loc[lp['element_type'], 'singular_name_short']
                    picks.append([
                        w, lp['web_name'], position, lp['element_type'], lp['name'], buy_price[p], sell_price[p], round(points_player_week[p,w],2), is_lineup, is_captain, is_vice, is_transfer_in, is_transfer_out
                    ])

        picks_df = pd.DataFrame(picks, columns=['week', 'name', 'pos', 'type', 'team', 'buy_price', 'sell_price', 'xP', 'lineup', 'captain', 'vicecaptain', 'transfer_in', 'transfer_out']).sort_values(by=['week', 'lineup', 'type', 'xP'], ascending=[True, False, True, True])
        total_xp = so.expr_sum((lineup[p,w] + captain[p,w]) * points_player_week[p,w] for p in players for w in gameweeks).get_value()
        week_xp = {w: so.expr_sum((lineup[p,w] + captain[p,w]) * points_player_week[p,w] for p in players) for w in gameweeks}


        # Writing summary
        summary_of_actions = f"\n==Solution #{it+1}==\n"
        for w in gameweeks:
            summary_of_actions += f"** GW {w}:\n"
            summary_of_actions += f"xP={round(week_xp[w].get_value(),3)}, ITB={in_the_bank[w].get_value()}, FT={free_transfers[w].get_value()}, PT={penalized_transfers[w].get_value()}\n"
            for p in players:
                if transfer_in[p,w].get_value() > 0.5:
                    summary_of_actions += f"Buy {p} - {merged_data['web_name'][p]}\n"
                if transfer_out[p,w].get_value() > 0.5:
                    summary_of_actions += f"Sell {p} - {merged_data['web_name'][p]}\n"
        print(summary_of_actions)

        next_gw_action = "Buy: "
        bought = []
        for p in players:
            if transfer_in[p, next_gw].get_value() > 0.5:
                bought.append(merged_data['web_name'][p])
        next_gw_action += ', '.join(bought)
        sold = []
        for p in players:
            if transfer_out[p, next_gw].get_value() > 0.5:
                sold.append(merged_data['web_name'][p])
        next_gw_action += " Sell: " + ', '.join(sold)    

        results.append({
            'iter': it+1,
            'picks': picks_df,
            'objective': -round(model.get_objective_value(),3),
            'next_gw_obj': round(gw_total[next_gw].get_value(),3),
            'summary': summary_of_actions,
            'total_xp': total_xp,
            'next_gw_xp': week_xp[next_gw].get_value(),
            'next_gw_action': next_gw_action
        })

        if it != nsols-1:
            if alternative_solution == '1gw-buy':
                actions = so.expr_sum(transfer_in[p, next_gw] for p in players if transfer_in[p, next_gw].get_value() > 0.5)
                gw_range = [next_gw]
            elif alternative_solution == 'horizon-buy':
                actions = so.expr_sum(transfer_in[p, w] for p in players for w in gameweeks if transfer_in[p, w].get_value() > 0.5)
                gw_range = gameweeks
            elif alternative_solution == '1gw-buy-sell':
                actions = so.expr_sum(transfer_in[p, next_gw] for p in players if transfer_in[p, next_gw].get_value() > 0.5) +\
                          so.expr_sum(transfer_out[p, next_gw] for p in players if transfer_out[p, next_gw].get_value() > 0.5)
                gw_range = [next_gw]
            elif alternative_solution == 'horizon-buy-sell':
                actions = so.expr_sum(transfer_in[p, w] for p in players for w in gameweeks if transfer_in[p, w].get_value() > 0.5) +\
                          so.expr_sum(transfer_out[p, w] for p in players for w in gameweeks if transfer_out[p, w].get_value() > 0.5)
                gw_range = gameweeks

            if actions.get_value() != 0:
                model.add_constraint(actions <= actions.get_value() - 1, name=f'cutoff_{it}')
            else:
                model.add_constraint(so.expr_sum(number_of_transfers[w] for w in gw_range) >= 1, name=f'cutoff_{it}')

        if strategy == '2-step':
            model.drop_constraint(tradeoff_con)

        try:
            pathlib.Path(f'gdrive/MyDrive/tmp/{problem_name}.mps').unlink()
            pathlib.Path(f'gdrive/MyDrive/tmp/{problem_name}_sol.txt').unlink()
        except:
            pass

    return {'model': model, 'results': results}

In [27]:
r = solve_decision_support(team_id=151180, gw=11, 
        options={
            'number_of_solutions': 5,
            'horizon':4,
            'objective':'decay', # you can use 'regular' or 'decay'
            'alternative_solution': '1gw-buy'} # you can use '1gw-buy', '1gw-buy-sell', 'horizon-buy', 'horizon-buy-sell'
        )
r['results']

NOTE: Initialized model ds_h4_NxRkL.
/bin/bash: command{}: command not found


FileNotFoundError: ignored

In [163]:
def multi_period_solver(team_id, gw, horizon, objective='regular', decay_base=0.84, team_override=False, team=[]):
    """
    Solves decision support problem for FPL
    Parameters
    ----------
    team_id: integer
        FPL ID of the team to be optimized
    gw: integer
        Upcoming (next) gameweek
    options: dict
        Options for the FPL problem
    """
    
    # load data
    data = get_data(team_id, gw-1)
    ft = get_rolling(team_id, gw - 1)[0] + 1
    merged_data = data['merged_data']
    team_data = data['team_data']
    type_data = data['type_data']
    next_gw = gw
    initial_squad = data['initial_squad']
    itb = data['itb']
    problem_name = f'mp_b{itb}_h{horizon}_o{objective}_d{decay_base}'
    
    # prepare data
    players = merged_data.index.to_list()
    element_types = type_data.index.to_list()
    teams = team_data['name'].to_list()
    gameweeks = list(range(next_gw, next_gw + horizon))
    all_gws = [next_gw-1] + gameweeks 
    # initialize solver
    prob = LpProblem(problem_name, LpMaximize)
    
    order = [0, 1, 2, 3]
    bench_weights = {0: 0.03, 1: 0.21, 2: 0.06, 3: 0.002}

    # set variables
    squad = LpVariable.dicts('squad', (players, all_gws), 0, 1, cat='Binary')
    lineup = LpVariable.dicts('lineup', (players, gameweeks), 0, 1, cat='Binary')
    captain = LpVariable.dicts('captain', (players, gameweeks), 0, 1, cat='Binary')
    vicecap = LpVariable.dicts('vicecap', (players, gameweeks), 0, 1, cat='Binary')
    transfer_in = LpVariable.dicts('transfer_in', (players, gameweeks), 0, 1, cat='Binary')
    transfer_out = LpVariable.dicts('transfer_out', (players, gameweeks), 0, 1, cat='Binary')
    in_the_bank = LpVariable.dicts('itb', all_gws, lowBound=0, cat="Continuous")
    free_transfers = LpVariable.dicts('ft', all_gws, lowBound=1, upBound=2, cat="Integer")
    penalized_transfers = LpVariable.dicts('pt', gameweeks, lowBound=0, cat="Integer")
    aux = LpVariable.dicts('aux', gameweeks, 0, 1, cat='Binary')
    bench = LpVariable.dicts('bench', (players, gameweeks, order), 0, 1, cat = 'Binary')
    # dictionaries
    lineup_type_count = {(t, w): lpSum(lineup[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    squad_type_count = {(t, w): lpSum(squad[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    player_type = merged_data['element_type'].to_dict()
    player_price = (merged_data['now_cost'] / 10).to_dict()
    sold_amount = {w: lpSum(player_price[p] * transfer_out[p][w] for p in players) for w in gameweeks}
    bought_amount = {w: lpSum(player_price[p] * transfer_in[p][w] for p in players) for w in gameweeks}
    points_player_week = {(p, w): merged_data.loc[p, f'{w}_Pts'] for p in players for w in gameweeks}
    squad_count = {w: lpSum(squad[p][w] for p in players) for w in gameweeks}
    lineup_count = {w: lpSum(lineup[p][w] for p in players) for w in gameweeks}
    captain_count = {w: lpSum(captain[p][w] for p in players) for w in gameweeks}
    vicecap_count = {w: lpSum(vicecap[p][w] for p in players) for w in gameweeks}
    number_of_transfers = {w: lpSum(transfer_out[p][w] for p in players) for w in gameweeks}
    number_of_transfers[next_gw-1] = 1
    transfer_diff = {w: number_of_transfers[w] - free_transfers[w] for w in gameweeks}
    # initial constraints
    for p in players:
      if p in initial_squad:
        prob += squad[p][next_gw-1] == 1
      else:
        prob += squad[p][next_gw-1] == 0
    prob += in_the_bank[next_gw-1] == itb
    prob += free_transfers[next_gw-1] == ft
    # set constraints

    for w in gameweeks:
        prob += squad_count[w] == 15
        prob += captain_count[w] == 1
        prob += vicecap_count[w] == 1
        prob += lpSum(lineup[p][w] for p in players) == 11
        prob += lpSum(bench[p][w][0] for p in players if player_type[p] == 1) == 1
        
        # free transfer constraints
        prob += free_transfers[w] == aux[w] + 1
        prob += free_transfers[w-1] - number_of_transfers[w] <= 2 * aux[w]
        prob += free_transfers[w-1] - number_of_transfers[w] >= aux[w] + (-14)*(1 - aux[w])
        prob += penalized_transfers[w] >= transfer_diff[w]
        prob += in_the_bank[w] == in_the_bank[w-1] + sold_amount[w] - bought_amount[w]
        
        for o in [1, 2, 3]:
          prob += lpSum(bench[p][w][o] for p in players) == 1
        for p in players:
            prob += squad[p][w] >= lineup[p][w]
            prob += lineup[p][w] >= captain[p][w]
            prob += lineup[p][w] >= vicecap[p][w]
            prob += captain[p][w] + vicecap[p][w] <= 1
            # transfer constraints
            prob += squad[p][w] == squad[p][w-1] + transfer_in[p][w] - transfer_out[p][w]
            prob += lpSum(bench[p][w][o] for o in order) + lineup[p][w] <= 1
            for o in order:
              prob += squad[p][w] >= bench[p][w][o]
        for t in element_types:
            prob += lineup_type_count[t, w] <= type_data.loc[t, 'squad_max_play']
            prob += lineup_type_count[t, w] >= type_data.loc[t, 'squad_min_play']
            prob += squad_type_count[t, w] == type_data.loc[t, 'squad_select']
        for t in teams:
            prob += lpSum(squad[p][w] for p in players if merged_data.loc[p, 'Team'] == t) <= 3
    # set objective
    gw_xp = {w: lpSum(points_player_week[p, w] * (lineup[p][w] + captain[p][w] + 0.1* vicecap[p][w] + lpSum(bench_weights[o] * bench[p][w][o] for o in order)) for p in players) for w in gameweeks}
    gw_total = {w: gw_xp[w] - 4 * penalized_transfers[w] for w in gameweeks}
    if objective == 'regular':
        total_xp = lpSum(gw_total[w] for w in gameweeks)
        prob += total_xp
    else:
        decay_objective = lpSum(gw_total[w] * pow(decay_base, w-next_gw) for w in gameweeks)
        prob += decay_objective
    # solve
    prob.writeLP(f'gdrive/MyDrive/{problem_name}.lp');
    prob.solve(PULP_CBC_CMD(msg=0));
    picks = []
    for w in gameweeks:
        for p in players:
            is_cap = 0
            is_vice = 0
            is_lineup = 0
            is_transfer_in = 0
            is_transfer_out = 0
            if squad[p][w].varValue + transfer_out[p][w].varValue == 1:
                if captain[p][w].varValue == 1:
                    is_cap = 1
                if vicecap[p][w].varValue == 1:
                    is_vice = 1
                if lineup[p][w].varValue == 1:
                    is_lineup = 1
                if transfer_in[p][w].varValue == 1:
                    is_transfer_in = 1
                if transfer_out[p][w].varValue == 1:
                    is_transfer_out = 1
                bench_value = -1
                for o in order:
                  if bench[p][w][o].varValue == 1:
                    bench_value = o
                picks.append([
                    w,
                    type_data.loc[merged_data.loc[p, 'element_type'], 'singular_name_short'],
                    merged_data.loc[p, 'element_type'],
                    merged_data.loc[p, 'web_name'],
                    merged_data.loc[p, 'name'],
                    player_price[p],
                    points_player_week[p, w], 
                    is_lineup, 
                    bench_value,
                    is_cap, 
                    is_vice,
                    is_transfer_in,
                    is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns = [
        'week',
        'pos', 
        'element_type', 
        'name', 
        'team', 
        'price', 
        'xPts', 
        'is_lineup', 
        'bench',
        'is_cap', 
        'is_vice',
        'is_transfer_in',
        'is_transfer_out'
    ]).sort_values(by=[
        'week', 
        'is_lineup', 
        'element_type', 
        'xPts'
    ], ascending=[
        True,
        False,
        True,
        True
    ])
    total_points = value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players for w in gameweeks))
    summary_of_actions = ""
    for w in gameweeks:
        summary_of_actions += f"** GW {w}:\n"
        summary_of_actions += f"ITB - {in_the_bank[w].varValue}, FT - {free_transfers[w].varValue}, PT - {penalized_transfers[w].varValue} , xPts - {round(value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players)), 2)}\n"
        for p in players:
            if transfer_in[p][w].varValue == 1:
                summary_of_actions += f"Buy {p} - {merged_data['web_name'][p]}\n"
            if transfer_out[p][w].varValue == 1:
                summary_of_actions += f"Sell {p} - {merged_data['web_name'][p]}\n"
    return {
        'picks': picks_df, 
        'total_xPts': total_points, 
        'summary': summary_of_actions
    }

In [164]:
sol = multi_period_solver(
    team_id=151180, 
    gw=12, 
    horizon = 1, 
    objective='regular', 
    team_override = False, 
    team = [559, 142, 237, 256, 491, 196, 96, 233, 141, 78, 205, 376, 413, 56, 115]
)
print(sol['summary'])

** GW 12:
ITB - 0.2, FT - 2.0, PT - 0.0 , xPts - 61.62
Sell 205 - Vardy
Buy 579 - Ronaldo



In [165]:
sol['picks']

,week,pos,element_type,name,team,price,xPts,is_lineup,bench,is_cap,is_vice,is_transfer_in,is_transfer_out
0,12,GKP,1,Ramsdale,Arsenal,4.7,3.197,1,-1,0,0,0,0
6,12,DEF,2,James,Chelsea,5.9,4.076,1,-1,0,0,0,0
10,12,DEF,2,Alexander-Arnold,Liverpool,7.7,5.408,1,-1,0,0,0,0
11,12,DEF,2,Cancelo,Man City,6.5,5.512,1,-1,0,0,0,0
8,12,MID,3,Raphinha,Leeds,6.6,3.699,1,-1,0,0,0,0
2,12,MID,3,Mbeumo,Brentford,5.6,4.277,1,-1,0,0,0,0
5,12,MID,3,Havertz,Chelsea,8.1,4.297,1,-1,0,0,0,0
9,12,MID,3,Salah,Liverpool,13.0,8.006,1,-1,1,0,0,0
15,12,FWD,4,Antonio,West Ham,8.2,4.226,1,-1,0,0,0,0
1,12,FWD,4,Toney,Brentford,6.7,4.634,1,-1,0,0,0,0


In [63]:
def multi_period_solver_obj(team_id, gw, options):
    """
    Solves decision support problem for FPL
    Parameters
    ----------
    team_id: integer
        FPL ID of the team to be optimized
    gw: integer
        Upcoming (next) gameweek
    options: dict
        Options for the FPL problem
    """
    
    # load data
    data = get_data(team_id, gw-1)
    ft = get_rolling(team_id, gw - 1)[0] + 1
    merged_data = data['merged_data']
    team_data = data['team_data']
    type_data = data['type_data']
    next_gw = gw
    initial_squad = data['initial_squad']
    itb = data['itb']

    horizon = options.get('horizon', min(3, 38-next_gw+1))
    objective = options.get('objective', 'regular')
    decay_base = options.get('decay_base', 0.84)
    nsols = options.get('number_of_solutions', 1)
    alternative_solution = options.get('alternative_solution', '1gw-buy')
    strategy = None
    problem_name = f'mp_b{itb}_h{horizon}_o{objective}_d{decay_base}'
    
    # prepare data
    players = merged_data.index.to_list()
    element_types = type_data.index.to_list()
    teams = team_data['name'].to_list()
    gameweeks = list(range(next_gw, next_gw + horizon))
    all_gws = [next_gw-1] + gameweeks 
    # initialize solver
    prob = LpProblem(problem_name, LpMaximize)
    
    # set variables
    squad = LpVariable.dicts('squad', (players, all_gws), 0, 1, cat='Binary')
    lineup = LpVariable.dicts('lineup', (players, gameweeks), 0, 1, cat='Binary')
    captain = LpVariable.dicts('captain', (players, gameweeks), 0, 1, cat='Binary')
    vicecap = LpVariable.dicts('vicecap', (players, gameweeks), 0, 1, cat='Binary')
    transfer_in = LpVariable.dicts('transfer_in', (players, gameweeks), 0, 1, cat='Binary')
    transfer_out = LpVariable.dicts('transfer_out', (players, gameweeks), 0, 1, cat='Binary')
    in_the_bank = LpVariable.dicts('itb', all_gws, lowBound=0, cat="Continuous")
    free_transfers = LpVariable.dicts('ft', all_gws, lowBound=1, upBound=2, cat="Integer")
    penalized_transfers = LpVariable.dicts('pt', gameweeks, lowBound=0, cat="Integer")
    aux = LpVariable.dicts('aux', gameweeks, 0, 1, cat='Binary')
    
    # dictionaries
    lineup_type_count = {(t, w): lpSum(lineup[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    squad_type_count = {(t, w): lpSum(squad[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    sell_price = (merged_data['SV']).to_dict()
    buy_price = (merged_data['BV']).to_dict()
    sold_amount = {w: lpSum(sell_price[p] * transfer_out[p][w] for p in players) for w in gameweeks}
    bought_amount = {w: lpSum(buy_price[p] * transfer_in[p][w] for p in players) for w in gameweeks}
    points_player_week = {(p, w): merged_data.loc[p, f'{w}_Pts'] for p in players for w in gameweeks}
    squad_count = {w: lpSum(squad[p][w] for p in players) for w in gameweeks}
    lineup_count = {w: lpSum(lineup[p][w] for p in players) for w in gameweeks}
    captain_count = {w: lpSum(captain[p][w] for p in players) for w in gameweeks}
    vicecap_count = {w: lpSum(vicecap[p][w] for p in players) for w in gameweeks}
    number_of_transfers = {w: lpSum(transfer_out[p][w] for p in players) for w in gameweeks}
    number_of_transfers[next_gw-1] = 1
    transfer_diff = {w: number_of_transfers[w] - free_transfers[w] for w in gameweeks}
    
    # initial constraints
 
    for p in initial_squad:
        prob += squad[p][next_gw-1] == 1 
    for p in players:
        if p not in initial_squad:
            prob += squad[p][next_gw-1] == 0
    prob += in_the_bank[next_gw-1] == itb
    prob += free_transfers[next_gw-1] == ft
    # set constraints
    for w in gameweeks:
        prob += squad_count[w] == 15
        prob += lineup_count[w] == 11 
        prob += captain_count[w] == 1
        prob += vicecap_count[w] == 1
        
        # free transfer constraints
        prob += free_transfers[w] == aux[w] + 1
        prob += free_transfers[w-1] - number_of_transfers[w] <= 2 * aux[w]
        prob += free_transfers[w-1] - number_of_transfers[w] >= aux[w] + (-14)*(1 - aux[w])
        prob += penalized_transfers[w] >= transfer_diff[w]
        prob += in_the_bank[w] == in_the_bank[w-1] + sold_amount[w] - bought_amount[w]
        
        for p in players:
            prob += squad[p][w] >= lineup[p][w]
            prob += lineup[p][w] >= captain[p][w]
            prob += lineup[p][w] >= vicecap[p][w]
            prob += captain[p][w] + vicecap[p][w] <= 1
            
            # transfer constraints
            prob += squad[p][w] == squad[p][w-1] + transfer_in[p][w] - transfer_out[p][w]
            
        for t in element_types:
            prob += lineup_type_count[t, w] <= type_data.loc[t, 'squad_max_play']
            prob += lineup_type_count[t, w] >= type_data.loc[t, 'squad_min_play']
            prob += squad_type_count[t, w] == type_data.loc[t, 'squad_select']
        for t in teams:
            prob += lpSum(squad[p][w] for p in players if merged_data.loc[p, 'Team'] == t) <= 3
    # set objective
    gw_xp = {w: lpSum(points_player_week[p, w] * (lineup[p][w] + captain[p][w] + 0.1* vicecap[p][w]) for p in players) for w in gameweeks}
    gw_total = {w: gw_xp[w] - 4 * penalized_transfers[w] for w in gameweeks}
    if objective == 'regular':
        total_xp = lpSum(gw_total[w] for w in gameweeks)
        prob += total_xp
    else:
        decay_objective = lpSum(gw_total[w] * pow(decay_base, w-next_gw) for w in gameweeks)
        prob += decay_objective
    # solve
    prob.writeLP(f'gdrive/MyDrive/{problem_name}.lp');
    prob.solve(PULP_CBC_CMD(msg=0));
    picks = []
    for w in gameweeks:
        for p in players:
            is_cap = 0
            is_vice = 0
            is_lineup = 0
            is_transfer_in = 0
            is_transfer_out = 0
            if squad[p][w].varValue + transfer_out[p][w].varValue == 1:
                if captain[p][w].varValue == 1:
                    is_cap = 1
                if vicecap[p][w].varValue == 1:
                    is_vice = 1
                if lineup[p][w].varValue == 1:
                    is_lineup = 1
                if transfer_in[p][w].varValue == 1:
                    is_transfer_in = 1
                if transfer_out[p][w].varValue == 1:
                    is_transfer_out = 1
                picks.append([
                    w,
                    type_data.loc[merged_data.loc[p, 'element_type'], 'singular_name_short'],
                    merged_data.loc[p, 'element_type'],
                    merged_data.loc[p, 'web_name'],
                    merged_data.loc[p, 'name'],
                    points_player_week[p, w], 
                    is_lineup, 
                    is_cap, 
                    is_vice,
                    is_transfer_in,
                    is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns=[
        'week',
        'pos', 
        'element_type', 
        'name', 
        'team', 
        'xPts', 
        'is_lineup', 
        'is_cap', 
        'is_vice',
        'is_transfer_in',
        'is_transfer_out'
     ]).sort_values(by=[
        'week', 
        'is_lineup', 
        'element_type', 
        'xPts'
    ], ascending=[
        True,
        False,
        True,
        True
    ])
    total_points = value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players for w in gameweeks))
    summary_of_actions = ""
    for w in gameweeks:
        summary_of_actions += f"** GW {w}:\n"
        summary_of_actions += f"ITB - {in_the_bank[w].varValue}, FT - {free_transfers[w].varValue}, PT - {penalized_transfers[w].varValue} , xPts - {round(value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players)), 2)}\n"
        for p in players:
            if transfer_in[p][w].varValue == 1:
                summary_of_actions += f"Buy {p} - {merged_data['web_name'][p]}\n"
            if transfer_out[p][w].varValue == 1:
                summary_of_actions += f"Sell {p} - {merged_data['web_name'][p]}\n"
    return {
        'picks': picks_df, 
        'total_xPts': total_points, 
        'summary': summary_of_actions
    }

In [66]:
sol = multi_period_solver_obj(team_id=151180, gw=12, 
        options={
            'number_of_solutions': 5,
            'horizon':4,
            'objective':'decay', # you can use 'regular' or 'decay'
            'alternative_solution': '1gw-buy'} # you can use '1gw-buy', '1gw-buy-sell', 'horizon-buy', 'horizon-buy-sell'
        )
print(sol['summary'])

** GW 12:
ITB - 1.6, FT - 2.0, PT - 0.0 , xPts - 60.92
Sell 141 - Havertz
Buy 265 - Foden
** GW 13:
ITB - 1.7, FT - 2.0, PT - 0.0 , xPts - 62.47
Sell 413 - Antonio
Buy 430 - Jiménez
** GW 14:
ITB - 0.2, FT - 2.0, PT - 0.0 , xPts - 57.74
Sell 205 - Vardy
Buy 357 - Kane
** GW 15:
ITB - 0.9, FT - 2.0, PT - 0.0 , xPts - 58.6
Sell 142 - James
Buy 370 - Reguilón

